In [1]:
!pip install transformers
!pip install torchmetrics

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchtext
from torchtext.legacy import data

import torchmetrics

from transformers import BertTokenizer, BertModel

import numpy as np

import time
import random
import functools

Установка seed для воспроизводимости результатов

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
MODE_RU = False #True - rus, False - multilanguage

# Подготовка данных

Импортируем токенизатор для BERT. Он определяет как текст в модели должен обрабатываться. Также он содержит словарь предобученной модели BERT. К сожалению, для русского языка нет отдельной предобученной модели и токенизатор, поэтому будем использовать многоязычную \
`bert-base-multilingual-uncased`.

Чтобы использовать предобученную модель нужно, чтобы словарь в точности совпадал со словарем предобученной модели.

In [5]:
if MODE_RU:
    tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
else:
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') 

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Нужно убедиться, что входная последовательность форматирована так же, как и предусмотрено в BERT

BERT была обучена на последовательностях, начинающихся с токена `[CLS]`

Пример преобразования:

```python
text = ['Я', 'пошел', 'в', 'тот', 'магазин']
```

```python
text = ['[CLS]', 'Я', 'пошел', 'в', 'тот', 'магазин']
```

Необходимо дополнительно убедиться, что неизвестный токен обозначается как `[UNK]`, а также токен выравнивания длины (padding token) обозначается как `[PAD]`

Получим специальные токены:

In [6]:
init_token = tokenizer.cls_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, pad_token, unk_token)

[CLS] [PAD] [UNK]


Получим индексы специальных токенов с помощью `convert_tokens_to_ids`

In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, pad_token_idx, unk_token_idx)

101 0 100


Необходимо, чтобы длина предложений во входных последовательностях не превосходила максимальной длины последовательностей в предобученной модели

In [8]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-multilingual-cased']

print(max_input_length)

512


Определим две вспомогательные функции для словарей

Первая будет 'подрезать' входные последовательности токенов до максимальной желаемой длины. Далее токены будут преобразовываться в индексы с помощью словаря. Она будет использоваться для последовательностей, для которых мы хотим найти теги.

Важно отметить, что послеодовательности будут подрезаться до max_length-1 из-за начального токена `[CLS]`

In [9]:
def cut_and_convert_to_id(tokens, tokenizer, max_input_length):
    tokens = tokens[:max_input_length-1]
    tokens = tokenizer.convert_tokens_to_ids(tokens)
    return tokens

Вторая вспомогательная функция просто подрезает последователньости до максимальной желаемой длины. Это используется для тегов. Мы не хотим передавать теги через словарь предобученной модели так как мы преимущественно используем русский язык. Будем строить словарь сами

In [10]:
def cut_to_max_length(tokens, max_input_length):
    tokens = tokens[:max_input_length-1]
    return tokens

Мы используем библиотеку `functools`, чтобы передать в функцию аргументы.

In [11]:
text_preprocessor = functools.partial(cut_and_convert_to_id,
                                      tokenizer = tokenizer,
                                      max_input_length = 64)

tag_preprocessor = functools.partial(cut_to_max_length,
                                     max_input_length = 64)

Дальше определим `Field`

Для поля `TEXT`, который будет обрабатывать последовательности, для которых нужно найти теги сделаем следующее:\
1) Не будем использовать словарь\
2) Приведем текст в нижний регистр, так как модель в нижнем регистре\
3) Предобработка будет проводиться функцией text_preprocessor\
4) Зададим индексы специальных токенов\

Для поля `UD_TAGS` нужно убедиться, что длина последовательности тегов соответствует длине текстовой последовательности. Так как мы добавляли `[CLS]` токен в начало текстовых последовательностей, то нужно проделать то же самое и с тегами. Добавим `<pad>` токен в начало и укажем модели не использовать его при подсчете метрик качества. У нас также не будет неизвестных тегов. Функция предобработки - `tag_preprocessor`.

In [12]:
TEXT = data.Field(use_vocab = False,
                  lower = False,
                  preprocessing = text_preprocessor,
                  init_token = init_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

UD_TAGS = data.Field(unk_token = None,
                     init_token = '<pad>',
                     preprocessing = tag_preprocessor)

Определим какие из созданны полей соотвествуют полям в датасете

In [13]:
fields = (("text", TEXT), ("udtags", UD_TAGS))

Загрузим датасет и сразу разделим его

In [14]:
dataset = data.TabularDataset(path='corp_cased.csv', format='csv', fields=fields, skip_header=True)

In [15]:
train_data, valid_data, test_data = dataset.split([0.8, 0.1, 0.1])

Пример

In [16]:
print(vars(train_data.examples[0]))

{'text': [14337, 100, 91765, 100, 100, 100, 100, 100, 100, 549, 100, 100, 100, 34848], 'udtags': ['VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'INFN', 'NOUN', 'PRTF', 'NOUN', 'CONJ', 'INFN', 'NOUN', 'ADJF', 'NOUN']}


Нужно построить словарь тегов. Сделаем это с помощью `.build_vocab` на `train_data`.

In [17]:
UD_TAGS.build_vocab(train_data)

print(UD_TAGS.vocab.stoi)

defaultdict(None, {'<pad>': 0, 'NOUN': 1, 'ADJF': 2, 'PREP': 3, 'VERB': 4, 'CONJ': 5, 'ADVB': 6, 'NPRO': 7, 'PRCL': 8, 'INFN': 9, 'PRTF': 10, 'Geox': 11, 'Name': 12, 'Surn': 13, 'ADJS': 14, 'PRTS': 15, 'NUMR': 16, 'GRND': 17, 'PRED': 18, 'COMP': 19, 'INTJ': 20, 'Patr': 21})


Определим итераторы. Они задают к порции данных будут подаваться во время обучения. Определим размер порции и также определим `device`, который автоматически будет подавать порции на GPU

BERT достаточно большая модель, поэтому размер порции сравнительно небольшой.

In [18]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort=False,
    batch_size = BATCH_SIZE,
    device = device)

# Построение модели

Модель довольно проста, все сложные моменты скрыты внутри BERT. Можем считать, что BERT - это такой эмбеддинг слой и все что нам нужно сделать - это добавить линейный слой

![](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/assets/pos-bert.png?raw=1)

В прошлом, желтые квадратики были эмбеддингами, которые создал соответствующий слой, но сейчас это эмбеддинги, которые мы получаем от предобученной модели BERT. Все входные значения подаются в BERT в одно и то же время. Стрелки между эмбеддингами BERT указывают, что эмбеддинги не считаются на для каждого токена индивидуально, а основываются на других токенах последовательности, что создает контекст.

Обратим внимание, что мы не задаем `embedding_dim` для нашей модели. Это размер выхода из предобученной модели и мы не можем менять его. Таким образом, мы просто получаем эту размерность из атрибута модели `hidden_size`

In [19]:
class BERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
  
        #text = [sent len, batch size]
    
        text = text.permute(1, 0)
        
        #text = [batch size, sent len]
        
        embedded = self.dropout(self.bert(text)[0])
        
        #embedded = [batch size, seq len, emb dim]
                
        embedded = embedded.permute(1, 0, 2)
                    
        #embedded = [sent len, batch size, emb dim]
        
        predictions = self.fc(self.dropout(embedded))
        
        #predictions = [sent len, batch size, output dim]
        
        return predictions

Далее, загрузим предобученную модель - прежде мы загружали только токенизатор модели.

In [20]:
if MODE_RU:
    bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')
else:
    bert = BertModel.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Обучение модели

Определим параметры для dropout слоя

In [21]:
OUTPUT_DIM = len(UD_TAGS.vocab)
DROPOUT = 0.25

model = BERTPoSTagger(bert,
                      OUTPUT_DIM, 
                      DROPOUT)

Посчитаем количество обучаемых параметров. Включаются параметры линейного слоя и все параметры BERT

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 177,870,358 trainable parameters


Определим оптимизатор. Обычно, при использовании предобученной модели мы используем коэффициент скорости обучения меньше, чем обычно. Это делается потому что мы не хотим радикально менять параметры, так как это может вызвать забывание того, что она выучила. Этот феномен называется катастрофическое забывание (catastrophic forgetting)

Выбрали 5е-5, так как это одно из значений, рекомендованных создателями модели. Могут быть и лучшие значения.

In [23]:
LEARNING_RATE = 5e-5

optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

Определим функцию потерь, убеждаясь, что игнорируем токены заполнения (pad)

In [24]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

Дальше, просто помещаемс модель на GPU

In [25]:
model = model.to(device)
criterion = criterion.to(device)

Определим функцию, которая подсчитываем точность предсказанных тегов, игнорируя токены заполнения

In [26]:
accuracy = torchmetrics.Accuracy(num_classes=len(UD_TAGS.vocab), ignore_index=TAG_PAD_IDX).to(device)
precision = torchmetrics.Precision(num_classes=len(UD_TAGS.vocab), ignore_index=TAG_PAD_IDX).to(device)
recall = torchmetrics.Recall(num_classes=len(UD_TAGS.vocab), ignore_index=TAG_PAD_IDX).to(device)
f1score = torchmetrics.F1Score(num_classes=len(UD_TAGS.vocab), ignore_index=TAG_PAD_IDX).to(device)

Определим `train` и `evaluate` функции для обучения и тестирования модели

In [27]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0

    model.train()
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.udtags
                
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        predictions = model(text)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
        
        accuracy(predictions, tags)
        precision(predictions, tags)
        recall(predictions, tags)
        f1score(predictions, tags)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    acc = accuracy.compute()
    f1 = f1score.compute()
    p = precision.compute()
    r = recall.compute()

    accuracy.reset()
    f1score.reset()
    precision.reset()
    recall.reset()
        
    return epoch_loss / len(iterator), acc, f1, p, r

In [28]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.udtags
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            accuracy.update(predictions, tags)
            precision(predictions, tags)
            recall(predictions, tags)
            f1score.update(predictions, tags)

            epoch_loss += loss.item()
        
        acc = accuracy.compute()
        f1 = f1score.compute()
        p = precision.compute()
        r = recall.compute()

        accuracy.reset()
        f1score.reset()
        precision.reset()
        recall.reset()
        
    return epoch_loss / len(iterator), acc, f1, p, r

Определим вспомогаетльную функцию для подсчета времени

In [29]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Наконец, можем обучить модели.

Эта модель обучается достаточно долго из-за большого количества параметров

In [30]:
N_EPOCHS = 3

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_f1, train_p, train_r = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc, valid_f1, valid_p, valid_r = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1: {train_f1:.2f} | Train precision: {train_p:.2f} | Train recall: {train_r:.2f}')
    print(f'\t Val Loss: {valid_loss:.3f} |  Val Acc: {valid_acc*100:.2f}% | Val F1: {valid_f1:.2f}  | Val precision: {valid_p:.2f} | Val recall: {valid_r:.2f}')

Epoch: 01 | Epoch Time: 24m 12s
	Train Loss: 0.716 | Train Acc: 77.81% | Train F1: 0.34 | Train precision: 0.22 | Train recall: 0.78
	 Val Loss: 0.613 |  Val Acc: 80.40% | Val F1: 0.35  | Val precision: 0.22 | Val recall: 0.80
Epoch: 02 | Epoch Time: 24m 9s
	Train Loss: 0.609 | Train Acc: 80.46% | Train F1: 0.36 | Train precision: 0.23 | Train recall: 0.80
	 Val Loss: 0.601 |  Val Acc: 80.63% | Val F1: 0.35  | Val precision: 0.23 | Val recall: 0.81
Epoch: 03 | Epoch Time: 24m 10s
	Train Loss: 0.573 | Train Acc: 81.40% | Train F1: 0.36 | Train precision: 0.23 | Train recall: 0.81
	 Val Loss: 0.604 |  Val Acc: 80.50% | Val F1: 0.35  | Val precision: 0.22 | Val recall: 0.81


Можем загрузить лучшую модель и попробовать ее на тестовом множестве

In [31]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc, test_f1, test_p, test_r = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}% | Test F1: {test_f1:.2f}  | Test precision: {test_p:.2f} | Test recall: {test_r:.2f}')

Test Loss: 0.605 |  Test Acc: 80.57% | Test F1: 0.35  | Test precision: 0.23 | Test recall: 0.81


# Вывод

Мы увидим как использовать модель для тегирования последовательностей.

Если мы подадим строку, то нужно ее разделить на индивидуальные токены. Мы сделаем это с помощью функции `tokenize` из `tokenizer`. После, перевести токены в числа так же, как мы делали раньше, используя `convert_tokens_to_ids`. Дальше добавляем `[CLS]` токен в начало последовательности

Дальше, подаем последовательность в модель и получаем предсказания для каждого токена. Отсекаем `[CLS]`, так как нам он не интересен.

In [32]:
def tag_sentence(model, device, sentence, tokenizer, text_field, tag_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        tokens = tokenizer.tokenize(sentence)
    else:
        tokens = sentence
    
    numericalized_tokens = tokenizer.convert_tokens_to_ids(tokens)
    numericalized_tokens = [text_field.init_token] + numericalized_tokens
        
    unk_idx = text_field.unk_token
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    predicted_tags = predicted_tags[1:]
        
    assert len(tokens) == len(predicted_tags)
    
    return tokens, predicted_tags, unks

Пример

In [33]:
sentence = 'Павел Дуров сегодня анонсировал создание нового офиса в Санкт-Петербурге'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence,
                                  tokenizer,
                                  TEXT, 
                                  UD_TAGS)

print(unks)

[]


In [34]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

Name		Павел
NOUN		Д
NOUN		##ур
Surn		##ов
ADVB		сегодня
CONJ		ан
CONJ		##он
VERB		##си
VERB		##ровал
NOUN		создание
ADJF		нового
PREP		о
NOUN		##фи
NOUN		##са
PREP		в
Geox		Санкт
ADJF		-
Geox		Петербурге
